# Array views

Code examples from [Think Complexity, 2nd edition](https://thinkcomplex.com).

Copyright 2019 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

This notebook explores the idea of array views by writing classes that implement slicing, reshaping, and transpose by creating views rather than creating or modifying arrays.

In [1]:
import numpy as np

### SliceView

Here's a class that represents a slice view of an array.

The `__init__` method just stores the "base array" and the parameters of the slice.

When we index into the `SliceView`, `__getitem__` uses the slice parameters to compute an index into the base array.

In [2]:
class SliceView:
    
    def __init__(self, base, slc):
        self.base = base
        self.start = slc.start
        self.stop = slc.stop
        self.step = slc.step if slc.step else 1
        
    def __getitem__(self, key):
        index = self.start + key * self.step
            
        if index < self.start:
            raise IndexError('too low')
            
        if index >= self.stop:
            raise IndexError('too high')
        
        return self.base[index]

As an example, here's a base array with integers from 0 to 100 (so the values in the array are the same as the indices).

In [3]:
a = np.arange(100)
a[10]

And here's a `SliceView` that selects the elements from index 10 to 20 (not including 20), in steps of 2.  

In [4]:
my_sv = SliceView(a, slice(10, 20, 2))
my_sv

Now here's the same thing using a NumPy slice index:

In [5]:
np_sv = a[10:20:2]
np_sv

We can test that my `SliceView` yields the same result as NumPy's:

In [6]:
assert my_sv[0] == np_sv[0]
my_sv[0]

In [7]:
assert my_sv[4] == np_sv[4]
my_sv[4]

And it generates an `IndexError` if we go out of bounds.

In [8]:
my_sv[5]

**Exercise:** So far, this implementation doesn't handle negative indices:

In [9]:
np_sv[-1]

In [10]:
my_sv[-1]

Make a modified version of `SliceView`, called `SliceViewNeg`, that handles the following test cases with negative indices.

In [11]:
# Solution goes here

Run the following cell to make a `SliceViewNeg`:

In [12]:
my_svn = SliceViewNeg(a, slice(10, 20, 2))
my_svn

Then run the following tests.

In [13]:
assert my_svn[-1] == np_sv[-1]
my_svn[-1]

In [14]:
assert my_svn[-5] == np_sv[-5]
my_svn[-5]

And the following cell should raise an `IndexError`.

In [15]:
my_svn[-6]

## Reshape

The following class defines `ReshapeView`, which makes it possible to view a 1D array as if it were 2D with the given shape (number of rows and columns).

In [16]:
class ReshapeView:
    
    def __init__(self, base, shape):
        self.base = base
        self.shape = shape
        
    def __getitem__(self, key):
        i, j = key
        nrows, ncols = self.shape
        index = i * ncols + j
        return self.base[index]
    
def reshape(array, shape):
    return ReshapeView(array, shape)

Here's a `ReshapeView` of the same base array, now with 5 rows and 20 columns.

In [17]:
my_rv = reshape(a, (5, 20))
my_rv

Here's a reshaped array using `np.reshape`.

In [18]:
np_rv = np.reshape(a, (5, 20))
np_rv

The following tests show that the two views yield the same results.

In [19]:
assert my_rv[0, 0] == np_rv[0, 0]
my_rv[0, 0]

In [20]:
assert my_rv[0, 19] == np_rv[0, 19]
my_rv[0, 19]

In [21]:
assert my_rv[4, 0] == np_rv[4, 0]
my_rv[4, 0]

In [22]:
assert my_rv[4, 19] == np_rv[4, 19]
my_rv[4, 19]

**Exercise:** The implementation so far does not check bounds on both axes, so if you go past the end of a row, it wraps around to the next row.

In [23]:
my_rv[0, 20]

Write a version of `ReshapeView`, called `ReshapeViewCheck`, that raises an exception if you go past the end of a row.

In [24]:
# Solution goes here

Run the following cell to create a `ReshapeViewCheck` and confirm that it raises an `IndexError` if you go past the end of a row.

In [25]:
def reshape_check(array, shape):
    return ReshapeViewCheck(array, shape)

my_rvc = reshape_check(a, (5, 20))
my_rvc[0, 20]

## Transpose

**Exercise:** Write a class called `TransposeView` that creates a transpose view of an array so it's consistent with `np.transpose`.  Your implementation should pass the tests below:

In [26]:
# Solution goes here

In [27]:
def transpose(view):
    return TransposeView(view)

In [28]:
my_tv = transpose(my_rv)
my_tv

In [29]:
np_tv = np.transpose(np_rv)
np_tv

In [30]:
assert my_tv[0, 0] == np_tv[0, 0]
my_tv[0, 0]

In [31]:
assert my_tv[0, 4] == np_tv[0, 4]
my_tv[0, 4]

In [32]:
assert my_tv[19, 0] == np_tv[19, 0]
my_tv[19, 0]

In [33]:
assert my_tv[19, 4] == np_tv[19, 4]
my_tv[19, 4]